In [3]:
!pip install cv2
!pip install playsound

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [2]:
import cv2
import numpy as np
import playsound
import smtplib

In [3]:
Fire_repoted = 0
Alarm_statues = False
Email_statues = True

video = cv2.VideoCapture('Burning_Toy_Car.mp4')

def play_audio():
    playsound.playsound("AlarmSound.mp3",True)
    
def send_email():
    
    recipientEmail = "Enter_Your_Email" 
    recipientEmail = recipientEmail.lower()

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.ehlo()
        server.starttls()
        server.login("Enter_Your_Email", 'Enter_Your_Email_Password')
        server.sendmail('Enter_Your_Email', recipientEmail, "Warning A Fire Accident has been reported on ABC Company")
        print("sent to {}".format(recipientEmail))
        server.close()
    except Exception as e:
        print(e)



while True:
    ret , frame = video.read() #ret equal false when there is no frame to read
        
    if ret == False:
        break
        
    frame = cv2.resize(frame,(1000,600))    
    blur = cv2.GaussianBlur(frame,(15,15),0) #remove some noises
    hsv = cv2.cvtColor(blur,cv2.COLOR_BGR2HSV) #convert RGB to HSV
    
    # RGB color boundaries
    lower = [18,120,120]
    upper = [35,255,255]
    
    #create NumPy arrays from the boundaries
    #8-bit unsigned int
    lower = np.array(lower , dtype = "uint8") 
    upper = np.array(upper , dtype = "uint8")
    
    #creat mask
    mask = cv2.inRange(hsv,lower,upper)
    
    output = cv2.bitwise_and(frame , hsv, mask=mask)    
    
    #enter the alarm
    size = cv2.countNonZero(mask)
    
    if int(size) > 15000:
        play_audio()
        Alarm_statues = True
        if Email_statues:
            send_email()
            Email_statues = False
        
    
    cv2.imshow("Output",output) 
    
    if cv2.waitKey(1) & 0xFF == ord("q"): #end while if someone clicks on q
        break

cv2.destroyAllWindows()
video.release()